# 1. Self 손글씨 맞추기 실습

MNIST 데이터셋을 사용해 학습된 인공신경망을 이용해 사용자가 제공한 숫자 이미지를 예측하는 코드를 작성해보겠습니다. 

MNIST 데이터셋은 손으로 쓴 숫자(0-9) 이미지들로 구성된 데이터셋

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from PIL import Image  # 사용자가 제공한 이미지를 처리하기 위해 사용

# MNIST 데이터셋 로드
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 데이터 전처리
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# 간단한 CNN 모델 생성
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))

# 사용자 이미지 예측 함수 정의
def predict_image(image_path):
    # 이미지 로드 및 전처리
    img = Image.open(image_path).convert('L')
    img = img.resize((28, 28))
    img = np.array(img).astype('float32') / 255
    img = 1 - img  # 흑백 반전 (흰 배경에 검은 숫자로 맞추기 위해)
    img = img.reshape((1, 28, 28, 1))
    
    # 예측
    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)
    print(f"예측된 숫자: {predicted_label}")

# 테스트할 사용자 이미지 경로
image_path = 'KakaoTalk_20241124_215514801.jpg'  # 여기서 'your_image.png'를 예측할 이미지 파일 경로로 변경하세요
predict_image(image_path)


C:\Users\JYB\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 17ms/step - accuracy: 0.8576 - loss: 0.4543 - val_accuracy: 0.9851 - val_loss: 0.0463
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.9820 - loss: 0.0549 - val_accuracy: 0.9894 - val_loss: 0.0296
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 16ms/step - accuracy: 0.9896 - loss: 0.0337 - val_accuracy: 0.9888 - val_loss: 0.0327
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9908 - loss: 0.0283 - val_accuracy: 0.9881 - val_loss: 0.0379
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - accuracy: 0.9927 - loss: 0.0239 - val_accuracy: 0.9892 - val_loss: 0.0338
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
예측된 숫자: 5


손글씨 코드설명

#### 1)  라이브러리 임포트

In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras import layers, models
# from tensorflow.keras.datasets import mnist
# from tensorflow.keras.utils import to_categorical
# from PIL import Image

#### 2) MNIST 데이터셋 로드

MNIST 데이터셋은 손으로 쓴 숫자 이미지 (0-9)로 구성되어 있으며, Keras에서 쉽게 로드할 수 있습니다.

(train_images, train_labels)는 학습용 데이터, (test_images, test_labels)는 테스트용 데이터를 의미합니다.

In [ ]:
#(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


#### 3) 데이터 전처리

In [ ]:
# train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
# test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
# train_labels = to_categorical(train_labels)
# test_labels = to_categorical(test_labels)
# 이미지를 (28, 28, 1)의 형태로 변형하고, float32 타입으로 변환하여 정규화(/ 255)합니다. 이는 픽셀 값을 0~1 사이로 맞추기 위함입니다.
# to_categorical() 함수는 레이블을 원-핫 인코딩으로 변환하여, 분류 문제에 적합하게 만들어줍니다.

#### 4) CNN 모델 생성

In [ ]:
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(10, activation='softmax'))

# Sequential 모델을 사용해 신경망을 구성합니다.
# Conv2D 층은 필터 크기가 (3, 3)인 컨볼루션 연산을 통해 이미지 특징을 추출합니다.
# MaxPooling2D는 풀링 연산을 통해 이미지 크기를 줄여 계산 효율성을 높입니다.
# Flatten()은 다차원 배열을 1차원으로 변환합니다.
# Dense 층은 완전 연결 계층으로, 마지막 층에서는 10개의 클래스(숫자 0-9)에 대한 확률을 출력합니다.

#### 5) 모델 컴파일

In [ ]:
# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
# adam 옵티마이저를 사용해 모델을 컴파일합니다.
# 손실 함수로는 categorical_crossentropy를 사용하며, 이는 다중 클래스 분류에 적합합니다.

#### 6) 모델 학습

In [ ]:
# model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))

# fit() 함수를 통해 모델을 학습시킵니다.
# 에포크 수는 5회, 배치 크기는 64로 설정하였습니다. validation_data는 학습 동안 모델의 성능을 평가하기 위해 사용됩니다.

#### 7) 사용자 이미지 예측 함수 정의

In [ ]:
# def predict_image(image_path):
#     # 이미지 로드 및 전처리
#     img = Image.open(image_path).convert('L')
#     img = img.resize((28, 28))
#     img = np.array(img).astype('float32') / 255
#     img = 1 - img  # 흑백 반전 (흰 배경에 검은 숫자로 맞추기 위해)
#     img = img.reshape((1, 28, 28, 1))
    
#     # 예측
#     prediction = model.predict(img)
#     predicted_label = np.argmax(prediction)
#     print(f"예측된 숫자: {predicted_label}")

In [ ]:
# predict_image() 함수는 사용자가 제공한 이미지를 모델에 입력하여 예측합니다.
# 이미지를 회색조(L)로 변환하고 (28, 28) 크기로 조정합니다.
# 1 - img를 통해 흑백을 반전하여 MNIST 데이터와 유사한 형식으로 만듭니다.
# 모델의 predict() 함수를 사용해 이미지를 예측하고, argmax()로 가장 높은 확률의 클래스를 출력합니다.

#### 8) 이미지 경로 설정 및 예측 실행

In [ ]:
# image_path = 'your_image.png'  # 여기서 'your_image.png'를 예측할 이미지 파일 경로로 변경하세요
# predict_image(image_path)

In [ ]:
# 'your_image.png'를 사용자가 예측하고자 하는 이미지 파일 경로로 변경한 후, predict_image() 함수를 호출해 예측 결과를 출력합니다.

# 2. 0부터 9까지의 확률로 보여주가 : epochs=5

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from PIL import Image
import os

# MNIST 데이터셋 로드
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 데이터 전처리
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# 간단한 CNN 모델 생성
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))

# 사용자 이미지 예측 함수 정의
def predict_image(image_path):
    if not os.path.exists(image_path):
        print(f"Error: 파일을 찾을 수 없습니다. 경로를 확인해주세요: {image_path}")
        return
    
    # 이미지 로드 및 전처리
    img = Image.open(image_path).convert('L')
    img = img.resize((28, 28))
    img = np.array(img).astype('float32') / 255
    img = 1 - img  # 흑백 반전 (흰 배경에 검은 숫자로 맞추기 위해)
    img = img.reshape((1, 28, 28, 1))
    
    # 예측
    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)
    print(f"예측된 숫자: {predicted_label}")
    print("0부터 9까지의 확률:")
    for i, prob in enumerate(prediction[0]):
        print(f"{i}: {prob * 100:.2f}%")

# 테스트할 사용자 이미지 경로
image_path = 'KakaoTalk_20241124_215514801.jpg'  # 여기서 'your_image.png'를 예측할 이미지 파일 경로로 변경하세요
predict_image(image_path)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 19s 18ms/step - accuracy: 0.8721 - loss: 0.4100 - val_accuracy: 0.9796 - val_loss: 0.0604
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9825 - loss: 0.0559 - val_accuracy: 0.9877 - val_loss: 0.0405
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.9884 - loss: 0.0366 - val_accuracy: 0.9872 - val_loss: 0.0419
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - accuracy: 0.9917 - loss: 0.0268 - val_accuracy: 0.9902 - val_loss: 0.0309
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9937 - loss: 0.0205 - val_accuracy: 0.9893 - val_loss: 0.0341
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
예측된 숫자: 7
0부터 9까지의 확률:
0: 7.08%
1: 13.14%
2: 10.42%
3: 6.38%
4: 8.49%
5: 9.13%
6: 5.68%
7: 30.87%
8: 3.04%
9: 5.75%


# 3. 0부터 9까지의 확률로 보여주가 : epochs=10

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from PIL import Image
import os

# MNIST 데이터셋 로드
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 데이터 전처리
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# 간단한 CNN 모델 생성
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_data=(test_images, test_labels))

# 사용자 이미지 예측 함수 정의
def predict_image(image_path):
    if not os.path.exists(image_path):
        print(f"Error: 파일을 찾을 수 없습니다. 경로를 확인해주세요: {image_path}")
        return
    
    # 이미지 로드 및 전처리
    img = Image.open(image_path).convert('L')
    img = img.resize((28, 28))
    img = np.array(img).astype('float32') / 255
    img = 1 - img  # 흑백 반전 (흰 배경에 검은 숫자로 맞추기 위해)
    img = img.reshape((1, 28, 28, 1))
    
    # 예측
    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)
    print(f"예측된 숫자: {predicted_label}")
    print("0부터 9까지의 확률:")
    for i, prob in enumerate(prediction[0]):
        print(f"{i}: {prob * 100:.2f}%")

# 테스트할 사용자 이미지 경로
image_path = 'KakaoTalk_20241124_215514801.jpg'  # 여기서 'your_image.png'를 예측할 이미지 파일 경로로 변경하세요
predict_image(image_path)


Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.8649 - loss: 0.4329 - val_accuracy: 0.9844 - val_loss: 0.0441
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.9826 - loss: 0.0560 - val_accuracy: 0.9876 - val_loss: 0.0429
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9878 - loss: 0.0398 - val_accuracy: 0.9879 - val_loss: 0.0349
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9905 - loss: 0.0297 - val_accuracy: 0.9896 - val_loss: 0.0308
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9936 - loss: 0.0210 - val_accuracy: 0.9910 - val_loss: 0.0293
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9942 - loss: 0.0177 - val_accuracy: 0.9913 - val_loss: 0.0273
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9948 - loss: 0.0161 - val_accuracy: 0.9898 - val_loss: 0.0314
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - accuracy: 0.9959 - loss: 0.0129 - 